In [2]:
!pip install -q -U tensorflow-addons

     |████████████████████████████████| 706kB 3.8MB/s 


# PART 1 Implement a Siamese network on the given dataset

In [3]:
# importing libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input
from keras.layers import Lambda 
from keras.layers import Dense
from keras.layers import Flatten
from keras import backend
from keras.models import Sequential
from keras.losses import BinaryCrossentropy
from sklearn.model_selection import train_test_split
import tensorflow_addons as tfa
import random
from tabulate import tabulate

In [4]:
# loading the dataset
(trainX, trainy), (testX, testy) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [5]:
trainX = trainX.astype('float32')/255.0
testX = testX.astype('float32')/255.0

In [6]:
trainX, valX, trainy, valy = train_test_split(trainX, trainy, test_size=0.2, shuffle=True, random_state=200)

In [7]:
# NUM_CLASSES = 10
# EPOCHS = 10
# INPUT_SHAPE = trainX.shape[1:]
# BATCH_SIZE=32
constvar_dec = {}
constvar_dec["NUM_CLASSES"] = 10 
constvar_dec["EPOCHS"] = 10
constvar_dec['INPUT_SHAPE'] = trainX.shape[1:]
constvar_dec['BATCH_SIZE'] = 32

In [8]:
class siamese_network:

  def __init__(self,var):
    self.var = var

  def euclidean_distance_output_shape(self, shape):
    shape1=[]
    shape2=[]
    shape1, shape2 = shapes
    return (shape1[0], 1)
    # return (shape[0][0],1)

  def contrastive_loss(self, y, y_pred):
    first_term = y*backend.square(y_pred) 
    second_term = (1-y)*backend.square(backend.maximum(1-y_pred,0))
    c_l = first_term + second_term
    return backend.mean(c_l)
    # return backend.mean(y*backend.square(y_pred) + (1-y)*backend.square(backend.maximum(1-y_pred,0)))

  def create_pairs(self, X, digit_idxs):
    # list for pairs of images and their labels
    pairs,labels = [],[]
    # go over each class
    for ci in range(self.var["NUM_CLASSES"]):
      # go over each image of a class
      for i in range(len(digit_idxs[ci])-1):
        # make a pair with 2 images of same class
        i1,i2 = digit_idxs[ci][i], digit_idxs[ci][i+1]
        pairs += [[X[i1], X[i2]]]
        labels += [1]
        # make a pair with 2 images of different classes
        # i1 = digit_idxs[ci][i]
        # the other class should be different
        diff_ci = random.randint(1,10)%10
        while diff_ci==ci:
          diff_ci = random.randint(1,10)%10
        i2 = random.randint(0, len(digit_idxs[diff_ci])-1)
        pairs += [[X[i1], X[i2]]]
        labels += [0]
    return np.array(pairs, dtype=np.float32), np.array(labels, dtype=np.float32)

  def met_accuracy(self, y, y_pred):
    acr = backend.equal(y, backend.cast(y_pred < 0.5, y.dtype))
    acr = backend.mean(acr)
    return acr

  def get_accuracy(self, model, pairs, y):
    y_pred = model.predict([pairs[:,0], pairs[:,1]])
    y_pred = y_pred.flatten()<0.5
    return np.mean(y_pred==y)

  def print_train_test_acc(self, model, train_pairs, test_pairs, train_y, test_y):
    acc = [self.get_accuracy(model, train_pairs, train_y),self.get_accuracy(model, test_pairs, test_y)]
    print('Train Set Accuracy: ',acc[0] )
    print('Test Set Accuracy: ', acc[1] )
    return acc

  def euclidean_distance(self, tensor):
    ed = backend.maximum(backend.sum(backend.square(tensor[0] - tensor[1]), axis=1, keepdims=True),backend.epsilon())
    ed = backend.sqrt(ed)
    return ed

  def get_model(self, model, loss, optimizer, acc_met, train_pairs, train_y, val_pairs, val_y):
    model.compile(loss=loss, optimizer=optimizer, metrics=[acc_met])
    model.fit(
              [train_pairs[:,0],
              train_pairs[:,1]],
              train_y,
              validation_data=([val_pairs[:,0],
                                val_pairs[:,1]],
                                val_y),
              batch_size=self.var["BATCH_SIZE"],
              epochs=self.var["EPOCHS"])
    return model

In [9]:
# get indexes for each class in train, test, val arrays
train_digit_idxs = [np.where(trainy == i)[0] for i in range(constvar_dec["NUM_CLASSES"])]
validate_digit_idxs = [np.where(valy == i)[0] for i in range(constvar_dec["NUM_CLASSES"])]
test_digit_idxs = [np.where(testy == i)[0] for i in range(constvar_dec["NUM_CLASSES"])]


In [10]:
siamese_net = siamese_network(constvar_dec)
# get a lot of pairs of images where each pair can have same type or different type
train_pairs, train_y = siamese_net.create_pairs(trainX, train_digit_idxs)
validate_pairs, validate_y = siamese_net.create_pairs(valX, validate_digit_idxs)
test_pairs, test_y = siamese_net.create_pairs(testX, test_digit_idxs)

In [11]:
base_model = Sequential([
                      Input(shape=constvar_dec["INPUT_SHAPE"]),
                      Flatten(),
                      Dense(256, activation='relu'),
                      Dense(256, activation='relu'),
                      Dense(256, activation='relu'),
                      Lambda(lambda  x: backend.l2_normalize(x,axis=1)),
])
input1 = Input(shape=constvar_dec["INPUT_SHAPE"])
input2 = Input(shape=constvar_dec["INPUT_SHAPE"])
processed1 = base_model(input1)
processed2 = base_model(input2)
distance = Lambda(siamese_net.euclidean_distance, output_shape=siamese_net.euclidean_distance_output_shape)([processed1, processed2])
model = Model([input1, input2], distance)

# PART 2 and PART 3 Use different loss functions with different optimisers

## Contrastive Loss

#### RMSprop

In [12]:
model = siamese_net.get_model(model, siamese_net.contrastive_loss, 'rmsprop', siamese_net.met_accuracy, train_pairs, train_y, validate_pairs, validate_y)

Epoch 1/10
3000/3000 [==============================] - 27s 9ms/step - loss: 0.1038 - met_accuracy: 0.8669 - val_loss: 0.0713 - val_met_accuracy: 0.9160
Epoch 2/10
3000/3000 [==============================] - 25s 8ms/step - loss: 0.0636 - met_accuracy: 0.9253 - val_loss: 0.0679 - val_met_accuracy: 0.9195
Epoch 3/10
3000/3000 [==============================] - 25s 8ms/step - loss: 0.0577 - met_accuracy: 0.9341 - val_loss: 0.0649 - val_met_accuracy: 0.9224
Epoch 4/10
3000/3000 [==============================] - 25s 8ms/step - loss: 0.0530 - met_accuracy: 0.9398 - val_loss: 0.0638 - val_met_accuracy: 0.9258
Epoch 5/10
3000/3000 [==============================] - 25s 8ms/step - loss: 0.0528 - met_accuracy: 0.9389 - val_loss: 0.0656 - val_met_accuracy: 0.9218
Epoch 6/10
3000/3000 [==============================] - 25s 8ms/step - loss: 0.0502 - met_accuracy: 0.9432 - val_loss: 0.0635 - val_met_accuracy: 0.9255
Epoch 7/10
3000/3000 [==============================] - 25s 8ms/step - loss: 0.049

In [13]:
acc11 = siamese_net.print_train_test_acc(model, train_pairs,test_pairs, train_y, test_y)

Train Set Accuracy:  0.9471035632423421
Test Set Accuracy:  0.9317817817817818


#### Adam


In [14]:
model = siamese_net.get_model(model, siamese_net.contrastive_loss, 'adam', siamese_net.met_accuracy, train_pairs, train_y, validate_pairs, validate_y)

Epoch 1/10
3000/3000 [==============================] - 22s 7ms/step - loss: 0.0491 - met_accuracy: 0.9432 - val_loss: 0.0624 - val_met_accuracy: 0.9267
Epoch 2/10
3000/3000 [==============================] - 21s 7ms/step - loss: 0.0462 - met_accuracy: 0.9464 - val_loss: 0.0633 - val_met_accuracy: 0.9260
Epoch 3/10
3000/3000 [==============================] - 21s 7ms/step - loss: 0.0456 - met_accuracy: 0.9461 - val_loss: 0.0648 - val_met_accuracy: 0.9240
Epoch 4/10
3000/3000 [==============================] - 21s 7ms/step - loss: 0.0442 - met_accuracy: 0.9474 - val_loss: 0.0642 - val_met_accuracy: 0.9259
Epoch 5/10
3000/3000 [==============================] - 22s 7ms/step - loss: 0.0433 - met_accuracy: 0.9480 - val_loss: 0.0652 - val_met_accuracy: 0.9243
Epoch 6/10
3000/3000 [==============================] - 22s 7ms/step - loss: 0.0426 - met_accuracy: 0.9482 - val_loss: 0.0628 - val_met_accuracy: 0.9284
Epoch 7/10
3000/3000 [==============================] - 22s 7ms/step - loss: 0.041

In [15]:
acc12 = siamese_net.print_train_test_acc(model, train_pairs, test_pairs, train_y, test_y)

Train Set Accuracy:  0.9527088976870182
Test Set Accuracy:  0.9303803803803804


#### Mini batch gradient descent

In [16]:
model = siamese_net.get_model(model, siamese_net.contrastive_loss, 'sgd', siamese_net.met_accuracy, train_pairs, train_y, validate_pairs, validate_y)

Epoch 1/10
3000/3000 [==============================] - 21s 7ms/step - loss: 0.0371 - met_accuracy: 0.9520 - val_loss: 0.0641 - val_met_accuracy: 0.9273
Epoch 2/10
3000/3000 [==============================] - 20s 7ms/step - loss: 0.0362 - met_accuracy: 0.9530 - val_loss: 0.0639 - val_met_accuracy: 0.9275
Epoch 3/10
3000/3000 [==============================] - 20s 7ms/step - loss: 0.0360 - met_accuracy: 0.9530 - val_loss: 0.0638 - val_met_accuracy: 0.9275
Epoch 4/10
3000/3000 [==============================] - 20s 7ms/step - loss: 0.0356 - met_accuracy: 0.9537 - val_loss: 0.0637 - val_met_accuracy: 0.9279
Epoch 5/10
3000/3000 [==============================] - 20s 7ms/step - loss: 0.0346 - met_accuracy: 0.9549 - val_loss: 0.0636 - val_met_accuracy: 0.9278
Epoch 6/10
3000/3000 [==============================] - 20s 7ms/step - loss: 0.0354 - met_accuracy: 0.9538 - val_loss: 0.0635 - val_met_accuracy: 0.9280
Epoch 7/10
3000/3000 [==============================] - 20s 7ms/step - loss: 0.036

In [17]:
acc13 = siamese_net.print_train_test_acc(model, train_pairs, test_pairs, train_y, test_y)

Train Set Accuracy:  0.9536153365284434
Test Set Accuracy:  0.9313313313313313


#### comparison

In [21]:
# !pip install tabulate
table = [['optimizer', 'ACCURACY'],
         ['RMS prop',str(acc11[0])],
         ['Adam',str(acc12[0])],
         ['Mini batch gradient descent',str(acc13[0])]]

print()
print()
print(tabulate(table, headers='firstrow'))



optimizer                      ACCURACY
---------------------------  ----------
RMS prop                       0.947104
Adam                           0.952709
Mini batch gradient descent    0.953615


## Cross entropy loss

#### RMSprop

In [22]:
model = siamese_net.get_model(model, BinaryCrossentropy(), 'rmsprop', siamese_net.met_accuracy, train_pairs, train_y, validate_pairs, validate_y)

Epoch 1/10
3000/3000 [==============================] - 27s 9ms/step - loss: 1.1693 - met_accuracy: 0.5709 - val_loss: 0.7311 - val_met_accuracy: 0.5120
Epoch 2/10
3000/3000 [==============================] - 26s 9ms/step - loss: 0.7038 - met_accuracy: 0.4650 - val_loss: 0.7267 - val_met_accuracy: 0.5043
Epoch 3/10
3000/3000 [==============================] - 26s 9ms/step - loss: 0.6853 - met_accuracy: 0.4338 - val_loss: 0.7283 - val_met_accuracy: 0.5066
Epoch 4/10
3000/3000 [==============================] - 26s 9ms/step - loss: 0.6743 - met_accuracy: 0.4152 - val_loss: 0.7318 - val_met_accuracy: 0.5084
Epoch 5/10
3000/3000 [==============================] - 25s 8ms/step - loss: 0.6633 - met_accuracy: 0.3960 - val_loss: 0.7276 - val_met_accuracy: 0.5005
Epoch 6/10
3000/3000 [==============================] - 25s 8ms/step - loss: 0.6552 - met_accuracy: 0.3852 - val_loss: 0.7353 - val_met_accuracy: 0.5069
Epoch 7/10
3000/3000 [==============================] - 26s 9ms/step - loss: 0.647

In [23]:
acc21 = siamese_net.print_train_test_acc(model, train_pairs, test_pairs, train_y, test_y)

Train Set Accuracy:  0.3369660345905397
Test Set Accuracy:  0.5535535535535535


#### Adam

In [24]:
model = siamese_net.get_model(model, BinaryCrossentropy(), 'adam', siamese_net.met_accuracy, train_pairs, train_y, validate_pairs, validate_y)

Epoch 1/10
3000/3000 [==============================] - 22s 7ms/step - loss: 0.6272 - met_accuracy: 0.3457 - val_loss: 0.7469 - val_met_accuracy: 0.5105
Epoch 2/10
3000/3000 [==============================] - 21s 7ms/step - loss: 0.6198 - met_accuracy: 0.3357 - val_loss: 0.7453 - val_met_accuracy: 0.5067
Epoch 3/10
3000/3000 [==============================] - 21s 7ms/step - loss: 0.6154 - met_accuracy: 0.3317 - val_loss: 0.7456 - val_met_accuracy: 0.5148
Epoch 4/10
3000/3000 [==============================] - 22s 7ms/step - loss: 0.6100 - met_accuracy: 0.3271 - val_loss: 0.7468 - val_met_accuracy: 0.5071
Epoch 5/10
3000/3000 [==============================] - 22s 7ms/step - loss: 0.6046 - met_accuracy: 0.3200 - val_loss: 0.7446 - val_met_accuracy: 0.5053
Epoch 6/10
3000/3000 [==============================] - 21s 7ms/step - loss: 0.5979 - met_accuracy: 0.3144 - val_loss: 0.7569 - val_met_accuracy: 0.5074
Epoch 7/10
3000/3000 [==============================] - 22s 7ms/step - loss: 0.592

In [25]:
acc22 = siamese_net.print_train_test_acc(model, train_pairs, test_pairs, train_y, test_y)

Train Set Accuracy:  0.2703375703271515
Test Set Accuracy:  0.5424924924924925


#### Mini batch gradient descent

In [26]:
model = siamese_net.get_model(model, BinaryCrossentropy(), 'sgd', siamese_net.met_accuracy, train_pairs, train_y, validate_pairs, validate_y)

Epoch 1/10
3000/3000 [==============================] - 21s 7ms/step - loss: 0.5630 - met_accuracy: 0.2722 - val_loss: 0.7952 - val_met_accuracy: 0.5044
Epoch 2/10
3000/3000 [==============================] - 20s 7ms/step - loss: 0.5536 - met_accuracy: 0.2734 - val_loss: 0.8001 - val_met_accuracy: 0.5043
Epoch 3/10
3000/3000 [==============================] - 20s 7ms/step - loss: 0.5502 - met_accuracy: 0.2702 - val_loss: 0.8050 - val_met_accuracy: 0.5045
Epoch 4/10
3000/3000 [==============================] - 20s 7ms/step - loss: 0.5468 - met_accuracy: 0.2682 - val_loss: 0.8042 - val_met_accuracy: 0.5034
Epoch 5/10
3000/3000 [==============================] - 20s 7ms/step - loss: 0.5474 - met_accuracy: 0.2705 - val_loss: 0.8153 - val_met_accuracy: 0.5029
Epoch 6/10
3000/3000 [==============================] - 20s 7ms/step - loss: 0.5467 - met_accuracy: 0.2689 - val_loss: 0.8191 - val_met_accuracy: 0.5037
Epoch 7/10
3000/3000 [==============================] - 20s 7ms/step - loss: 0.545

In [27]:
acc23 = siamese_net.print_train_test_acc(model, train_pairs, test_pairs, train_y, test_y)

Train Set Accuracy:  0.2665034382162951
Test Set Accuracy:  0.5383883883883884


#### comparison

In [28]:
table = [['optimizer', 'ACCURACY'],
         ['RMS prop',str(acc21[0])],
         ['Adam',str(acc22[0])],
         ['Mini batch gradient descent',str(acc23[0])]]

print()
print()
print(tabulate(table, headers='firstrow'))



optimizer                      ACCURACY
---------------------------  ----------
RMS prop                       0.336966
Adam                           0.270338
Mini batch gradient descent    0.266503


## Triplet Loss

#### RMSprop

In [29]:
model = siamese_net.get_model(model, tfa.losses.TripletSemiHardLoss(), 'rmsprop', siamese_net.met_accuracy, train_pairs, train_y, validate_pairs, validate_y)

Epoch 1/10
3000/3000 [==============================] - 35s 11ms/step - loss: 0.9735 - met_accuracy: 0.3875 - val_loss: 0.9790 - val_met_accuracy: 0.5048
Epoch 2/10
3000/3000 [==============================] - 32s 11ms/step - loss: 0.9710 - met_accuracy: 0.4646 - val_loss: 0.9790 - val_met_accuracy: 0.5036
Epoch 3/10
3000/3000 [==============================] - 31s 10ms/step - loss: 0.9699 - met_accuracy: 0.4730 - val_loss: 0.9797 - val_met_accuracy: 0.5024
Epoch 4/10
3000/3000 [==============================] - 31s 10ms/step - loss: 0.9688 - met_accuracy: 0.4814 - val_loss: 0.9802 - val_met_accuracy: 0.5010
Epoch 5/10
3000/3000 [==============================] - 31s 10ms/step - loss: 0.9686 - met_accuracy: 0.4824 - val_loss: 0.9798 - val_met_accuracy: 0.5019
Epoch 6/10
3000/3000 [==============================] - 31s 10ms/step - loss: 0.9681 - met_accuracy: 0.4854 - val_loss: 0.9804 - val_met_accuracy: 0.5020
Epoch 7/10
3000/3000 [==============================] - 31s 10ms/step - loss

In [30]:
acc31 = siamese_net.print_train_test_acc(model, train_pairs, test_pairs, train_y, test_y)

Train Set Accuracy:  0.4927797457803709
Test Set Accuracy:  0.5142642642642643


#### Adam

In [31]:
model = siamese_net.get_model(model, tfa.losses.TripletSemiHardLoss(), 'adam', siamese_net.met_accuracy, train_pairs, train_y, validate_pairs, validate_y)

Epoch 1/10
3000/3000 [==============================] - 27s 9ms/step - loss: 0.9641 - met_accuracy: 0.4928 - val_loss: 0.9803 - val_met_accuracy: 0.5020
Epoch 2/10
3000/3000 [==============================] - 27s 9ms/step - loss: 0.9639 - met_accuracy: 0.4909 - val_loss: 0.9800 - val_met_accuracy: 0.5029
Epoch 3/10
3000/3000 [==============================] - 26s 9ms/step - loss: 0.9633 - met_accuracy: 0.4922 - val_loss: 0.9802 - val_met_accuracy: 0.5015
Epoch 4/10
3000/3000 [==============================] - 27s 9ms/step - loss: 0.9623 - met_accuracy: 0.4951 - val_loss: 0.9808 - val_met_accuracy: 0.5024
Epoch 5/10
3000/3000 [==============================] - 27s 9ms/step - loss: 0.9611 - met_accuracy: 0.4997 - val_loss: 0.9801 - val_met_accuracy: 0.5015
Epoch 6/10
3000/3000 [==============================] - 26s 9ms/step - loss: 0.9603 - met_accuracy: 0.4952 - val_loss: 0.9808 - val_met_accuracy: 0.5023
Epoch 7/10
3000/3000 [==============================] - 26s 9ms/step - loss: 0.958

In [32]:
acc32 = siamese_net.print_train_test_acc(model, train_pairs, test_pairs, train_y, test_y)

Train Set Accuracy:  0.4994790581371119
Test Set Accuracy:  0.5118118118118118


#### Mini batch gradient descent

In [33]:
model = siamese_net.get_model(model, tfa.losses.TripletSemiHardLoss(), 'sgd', siamese_net.met_accuracy, train_pairs, train_y, validate_pairs, validate_y)

Epoch 1/10
3000/3000 [==============================] - 26s 8ms/step - loss: 0.9493 - met_accuracy: 0.4948 - val_loss: 0.9808 - val_met_accuracy: 0.5018
Epoch 2/10
3000/3000 [==============================] - 25s 8ms/step - loss: 0.9421 - met_accuracy: 0.5024 - val_loss: 0.9810 - val_met_accuracy: 0.5016
Epoch 3/10
3000/3000 [==============================] - 25s 8ms/step - loss: 0.9393 - met_accuracy: 0.5016 - val_loss: 0.9809 - val_met_accuracy: 0.5017
Epoch 4/10
3000/3000 [==============================] - 25s 8ms/step - loss: 0.9387 - met_accuracy: 0.5001 - val_loss: 0.9809 - val_met_accuracy: 0.5017
Epoch 5/10
3000/3000 [==============================] - 25s 8ms/step - loss: 0.9376 - met_accuracy: 0.5013 - val_loss: 0.9808 - val_met_accuracy: 0.5014
Epoch 6/10
3000/3000 [==============================] - 25s 8ms/step - loss: 0.9373 - met_accuracy: 0.5006 - val_loss: 0.9809 - val_met_accuracy: 0.5016
Epoch 7/10
3000/3000 [==============================] - 25s 8ms/step - loss: 0.936

In [34]:
acc33 = siamese_net.print_train_test_acc(model, train_pairs, test_pairs, train_y, test_y)

Train Set Accuracy:  0.5014377995415712
Test Set Accuracy:  0.5092092092092092


#### comparison

In [35]:
table = [['optimizer', 'ACCURACY'], ['RMS prop',str(acc21[0])], ['Adam',str(acc22[0])], ['Mini batch gradient descent',str(acc31[1])]]
print()
print()
print(tabulate(table, headers='firstrow'))



optimizer                      ACCURACY
---------------------------  ----------
RMS prop                       0.336966
Adam                           0.270338
Mini batch gradient descent    0.514264


### comparison of three losses

In [36]:
table = [['Loss Function', 'ACCURACY'], ['Contrastive',str(acc13[0])], ['Cross entropy',str(acc21[1])], ['tripple loss',str(acc23[0])]]

print()
print()
print(tabulate(table, headers='firstrow'))



Loss Function      ACCURACY
---------------  ----------
Contrastive        0.953615
Cross entropy      0.553554
tripple loss       0.266503


# PART 4

## Hyper Parameter Optimization to improve the accuracy

In [37]:
# reducing batcch size
constvar_dec2 = {}
constvar_dec2["NUM_CLASSES"] = 10 
constvar_dec2["EPOCHS"] = 10
constvar_dec2['INPUT_SHAPE'] = trainX.shape[1:]
constvar_dec2['BATCH_SIZE'] = 16
siamese_net2 = siamese_network(constvar_dec2)

In [39]:
# tuning hyperparameters of adam optimiser
opt = Adam(lr=0.01, beta_1=0.9)
model = siamese_net2.get_model(model, siamese_net2.contrastive_loss,optimizer=opt, siamese_net2.met_accuracy, train_pairs, train_y, validate_pairs, validate_y)

Epoch 1/10
5999/5999 [==============================] - 44s 7ms/step - loss: 0.1643 - met_accuracy: 0.7701 - val_loss: 0.0728 - val_met_accuracy: 0.9177
Epoch 2/10
5999/5999 [==============================] - 43s 7ms/step - loss: 0.0602 - met_accuracy: 0.9334 - val_loss: 0.0663 - val_met_accuracy: 0.9215
Epoch 3/10
5999/5999 [==============================] - 43s 7ms/step - loss: 0.0499 - met_accuracy: 0.9435 - val_loss: 0.0666 - val_met_accuracy: 0.9229
Epoch 4/10
5999/5999 [==============================] - 43s 7ms/step - loss: 0.0465 - met_accuracy: 0.9462 - val_loss: 0.0653 - val_met_accuracy: 0.9228
Epoch 5/10
5999/5999 [==============================] - 44s 7ms/step - loss: 0.0432 - met_accuracy: 0.9491 - val_loss: 0.0651 - val_met_accuracy: 0.9247
Epoch 6/10
5999/5999 [==============================] - 46s 8ms/step - loss: 0.0441 - met_accuracy: 0.9473 - val_loss: 0.0653 - val_met_accuracy: 0.9252
Epoch 7/10
5999/5999 [==============================] - 52s 9ms/step - loss: 0.040

In [40]:
print('Hyperparameter Tuned:- Reduced batch size, Changed Learning Rate, Added beta_1 for Adam optimiser')
acc = siamese_net2.print_train_test_acc(model, train_pairs, test_pairs, train_y, test_y)

Hyperparameter Tuned:- Reduced batch size, Changed Learning Rate, Added beta_1 for Adam optimiser
Train Set Accuracy:  0.952542196290894
Test Set Accuracy:  0.9268768768768769


# PART 5

## Pros of Siamese Networks
1. It only requires a few training examples of each class to recognize samples of that class.
2. It Can learn semantic similarity

## Cons of Siamese Networks
1. It needs more training time as it operates on pairs of training samples per class. 
2. It provides output in the form of distance from each class, not the probability of a test sample to belong to a class.